In [12]:
import os
from os import mkdir
from os.path import join,exists,basename,splitext
from glob import glob
import cv2
import re
import matplotlib.pyplot as plt
import numpy as np

In [13]:
path_root = os.getcwd()
path_videos = join(path_root, "videos_processed")
path_frames = join(path_root,"videos_frames")
path_crop_frames = join(path_root,"videos_frames_crop")

if not exists(path_crop_frames):
    mkdir(path_crop_frames)

In [16]:
def show_image(image, cmap=None):
    plt.figure(figsize=(20,15))
    if cmap is not None:
        plt.imshow(image, cmap="gray")
    else:
        plt.imshow(image)
    plt.show()
    
    
def get_max_contour(image_bin):
    contours,_ = cv2.findContours(image_bin,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    max_area = 0
    i_max_area = -1
    for i,contour in enumerate(contours):
        x,y,w,h = cv2.boundingRect(contour)
        area = w*h
        if area > max_area:
            max_area = area
            i_max_area = i

    return cv2.boundingRect(contours[i_max_area])


def remove_black_border(image_rgb):
    image_gray = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2GRAY)

    _,thresh = cv2.threshold(image_gray,1,255,cv2.THRESH_BINARY)
    kernel = np.ones((7,7),np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=5)
    x,y,w,h = get_max_contour(thresh)                

    image_crop = image_rgb[y:y+h,x:x+w,:]
    return image_crop

In [31]:
list_path_frames = glob(join(path_frames,"*.png"))


for path_frame in list_path_frames:
    filename = basename(path_frame)
    filename_image_crop = join(path_crop_frames,filename)
    
    if not exists(filename_image_crop):  
        image_rgb = cv2.imread(path_frame,1)
        image_crop = remove_black_border(image_rgb)        
        cv2.imwrite(filename_image_crop,image_crop)